In [9]:
import pandas as pd
import numpy as np

In [10]:
# Chargement des DataSets
heart_df = pd.read_csv('../../../Data/Classification/Indicators of Heart Disease/heart_2022_no_nans_clean.csv')

In [11]:
def target_class_balance(df, imbalance_levels, target_column):
    df_dict = {}

    for imbalance_factor in imbalance_levels:
        df_polluted = df.copy()

        # Vérifier que le facteur de déséquilibre est entre 0 et 1
        if not 0 <= imbalance_factor <= 1:
            raise ValueError("Le facteur de déséquilibre doit être compris entre 0 et 1.")

        # Compter le nombre d'occurrences de chaque classe
        class_counts = df_polluted[target_column].value_counts()
        min_class = class_counts.idxmin()  # Classe minoritaire
        max_class = class_counts.idxmax()  # Classe majoritaire

        # Calculer le nombre de lignes à ajouter ou retirer
        total_samples = len(df_polluted)

        new_class_counts = {}
        for class_label, count in class_counts.items():
            if class_label == max_class:
                # Réduire le nombre de la classe majoritaire
                new_class_counts[class_label] = int(count * imbalance_factor)
            else:
                # Augmenter le nombre des autres classes
                new_class_counts[class_label] = count + int((total_samples - count) * (1 - imbalance_factor))

        # Créer le DataFrame pollué avec le nouvel équilibre des classes
        df_balanced = pd.DataFrame()
        for class_label, new_count in new_class_counts.items():
            class_df = df_polluted[df_polluted[target_column] == class_label]
            if new_count > len(class_df):
                # Répéter les lignes pour augmenter le nombre
                repeats = int(np.ceil(new_count / len(class_df)))
                df_balanced = pd.concat([df_balanced, pd.concat([class_df] * repeats).iloc[:new_count]], ignore_index=True)
            else:
                # Échantillonner les lignes pour réduire le nombre
                df_balanced = pd.concat([df_balanced, class_df.sample(new_count, random_state=42)], ignore_index=True)

        # Ajouter le DataFrame déséquilibré au dictionnaire avec le facteur de déséquilibre comme clé
        df_dict[f"{int(imbalance_factor * 100)}%"] = df_balanced

    return df_dict

In [12]:
# Liste des pourcentages de valeurs manquantes à introduire
imbalance_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# Génération des DataFrames avec valeurs manquantes
heart_df_with_target_class_balance = target_class_balance(heart_df, imbalance_levels,'HadHeartAttack')

In [13]:
for key, value in heart_df_with_target_class_balance.items():
    value.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Target Class Balance/heart_{key}.csv", index=False)